In [1]:
using IPUToolkit.IPUCompiler, IPUToolkit.Poplar

This notebook shows a small example of a very simple DSL that you can use to

In [2]:
# Define the arrays that will be used during the program.  `input` is a host array that will
# be automatically copied to an IPU array, the other `PoplarVector`s are placeholders for
# IPU arrays that will be populated during the execution of the program.
input = Float32[5, 2, 10, 102, -10, 2, 256, 15, 32, 100]
outvec1 = PoplarVector{Float32}(undef, 10)
outvec2 = PoplarVector{Float32}(undef, 10)
outvec3 = PoplarVector{Float32}(undef, 10)

# Get the device.
device = Poplar.get_ipu_device()

# Inside `@ipuprogram` you can do only the following things:
#
# * define functions, which will be used as codelets in the IPU program
# * call these functions, which will automatically build the graph of the calls for you
# * print tensors on the IPU with the "special" function `print_tensor`
# * copy IPU tensors to the host
@ipuprogram device begin
    # Define the functions/codelets.  All arguments must be `VertexVector`s.
    function TimesTwo(inconst::VertexVector{Float32, IPUCompiler.In}, outvec::VertexVector{Float32, IPUCompiler.Out})
        outvec .= 2 .* inconst
    end
    function Sort(invec::VertexVector{Float32, IPUCompiler.In}, outvec::VertexVector{Float32, IPUCompiler.Out})
        outvec .= invec
        sort!(outvec)
    end
    function Sin(invec::VertexVector{Float32, IPUCompiler.In}, outvec::VertexVector{Float32, IPUCompiler.Out})
        for idx in eachindex(outvec)
            @inbounds outvec[idx] = sin(invec[idx])
        end
    end

    # Run the functions.  Arguments must be the arrays defined above, either host arrays
    # (which will be copied to the IPU automatically) or `PoplarVector`s.
    TimesTwo(input, outvec1)
    Sort(outvec1, outvec2)
    Sin(outvec2, outvec3)

    # `print_tensor` is a special function which prints tensors to the host
    # using `Poplar.ProgramPrintTensor` under the hood.  Syntax is
    #     print_tensor(<LABEL>, <tensor variable>)
    print_tensor("Input",    input)
    print_tensor("TimesTwo", outvec1)
    print_tensor("Sorted",   outvec2)
    print_tensor("Sin",      outvec3)

    # Copy IPU tensors to the host.  The right-hand side must be one of the tensors defined
    # above, the left-hand side is the name of a host array which will be created
    # automatically for you, so you will be able to reference them after the `@ipuprogram`.
    jl_outvec1 = outvec1
    jl_outvec2 = outvec2
    jl_outvec3 = outvec3
end

# Detach the device when we're done.
Poplar.detach_devices()

[ Info: Trying to attach to device 0...
[ Info: Successfully attached to device 0
✓ Compiling codelet TimesTwo: 	 Time: 0:00:05
✓ Compiling codelet Sort: 	 Time: 0:00:01
✓ Compiling codelet Sin: 	 Time: 0:00:00
Input: {5,2,10,102,-10,2,256,15,32,100}
TimesTwo: {10,4,20,204,-20,4,512,30,64,200}
Sorted: {-20,4,4,10,20,30,64,200,204,512}
Sin: {-0.912945,-0.756802,-0.756802,-0.544021,0.912945,-0.988032,0.920026,-0.873297,0.20212,0.0795185}


In [3]:
jl_outvec1 ≈ 2 .* input

true

In [4]:
jl_outvec2 ≈ sort(jl_outvec1)

true

In [5]:
jl_outvec3 ≈ sin.(jl_outvec2)

true

In [6]:
# Always remember to release the device after use!
Poplar.detach_devices()